**Imports**

In [1]:
import csv
import sqlite3

import pandas as pd


**SQLite3 database Connection**

In [2]:
connection = sqlite3.connect("./S30 ETL Assignment.db")
cursor = connection.cursor()

**2. extracting the total quantities of each item bought per customer aged 18-35.**
 - *For each customer, get the sum of each item.*
 - *Items with no purchase (total quantity=0) should be omitted from the final list*
 - *No decimal points allowed (The company doesn’t sell half of an item)*
### SQL Solution

In [3]:
cursor.execute("""
SELECT
    s.customer_id Customer,
    c.age Age,
    i.item_name Item,
    SUM(o.quantity) Quantity
FROM
    sales s
    LEFT JOIN customers c ON s.customer_id = c.customer_id
    LEFT JOIN orders o ON s.sales_id = o.sales_id
    LEFT JOIN items i ON o.item_id = i.item_id
WHERE
    c.age BETWEEN 18 AND 35
    AND o.quantity IS NOT NULL
GROUP BY
    c.customer_id,
    i.item_name
HAVING
    SUM(o.quantity) > 0
ORDER BY
    c.customer_id,
    i.item_name,
    c.age
""")
sql_result = cursor.fetchall()

### Pandas Solution

**Creating DataFrames**

In [4]:
sales_df = pd.read_sql_query("SELECT * FROM sales", connection)
orders_df = pd.read_sql_query("SELECT * FROM orders", connection)
items_df = pd.read_sql_query("SELECT * FROM items", connection)
customers_df = pd.read_sql_query("SELECT * FROM customers", connection)

**Combining the data**

In [5]:
combined_df = (customers_df.merge(sales_df, on="customer_id").merge(
    orders_df, on="sales_id").merge(items_df, on="item_id"))

**Filtering combined_df to fetch customer age between 18 & 35 AND Quantity without NULL values**

In [6]:
filtered_df = (combined_df[(combined_df["age"].between(18, 35))
                           & (combined_df["quantity"].notna())])

**Calculating the Quantity**

In [7]:
final_df = filtered_df.groupby(["customer_id", "age", "item_name"],
                               as_index=False)["quantity"].sum()
final_df = final_df[final_df["quantity"] > 0]
final_df["quantity"] = final_df["quantity"].astype(int)
final_df.rename(columns={
    "customer_id": "Customer",
    "age": "Age",
    "item_name": "Item",
    "quantity": "Quantity"
},
                inplace=True)

**Store the SQL result in a .csv file**

In [8]:
with open("./results/sql_output.csv", "w", newline='') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(["Customer", "Age", "Item", "Quantity"])  # Header
    for row in sql_result:
        writer.writerow(row)

**Store the Pandas result in a .csv file**

In [9]:
final_df.to_csv("./results/pandas_output.csv", sep=";", index=False)

**Closing the database connection**

In [10]:
connection.close()